In [1]:
pip install folium


   ---------------------------------------- 2/2 [folium]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import folium
import pandas as pd

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon


## Task 1: Mark all launch sites on a map

In [12]:
import folium
from folium.features import DivIcon

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#8B0000',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#8B0000;"><b>%s</b></div>' % 'NASA JSC',
    )
)

site_map.add_child(circle)
site_map.add_child(marker)

site_map

# COMMENTS SAID BLUE CIRCLE BUT LAB INSTRUCTIONS WERE FOR IT TO BE YELLOW T___T I CHANGED IT TO BLUE. IT LOOKED RIDICULOUS. CHANGED IT TO RED because it needs to pop off the map!

In [16]:
# Now, let's add a circle for each launch site in data frame launch_sites.

for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
# An example of folium.Circle:
#    folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))
    circle = folium.Circle(
        coordinate,
        radius=1000,
        color='#8B0000',   # red circle with a nice contrast to the suggested blue or yellow >_<
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
# An example of folium.Marker:
#    folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))    
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:##8B0000;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
# Added the equator line bacause of the question in the lab. I had no idea how close they are or aren't. -o-
folium.PolyLine(
    locations=[[0, -180], [0, 180]],
    color='green',
    weight=2,
    dash_array='5, 5'
).add_to(site_map)

site_map


# Observations:

 - The launch sites are in the cities located along the coast. This might be because of safety and accessibilty reasons.
 - The launch sites are in the proximity of the equator.
 - The locations are where climate is classified as mediterranean(Csb), so the decision in choosing launch sites might have something to do with the        weather.
 - The launch sites are also in close proximity to space centers/facilites. This provides support advantages
 - There are also airports near the launch sites. This might be for logistic support for materials and personel, and can also be used to control the air    space.
 - They also seemed to be situated near a clearing, reserves, or parks. This might be due to safety reasons, specially in the cities.
 - Military bases are also in close proximty with the launch sites. This might be for defensive measures, and air space control.

# Task 2: Mark the success/failed launches for each site on the map

In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [18]:
from folium.plugins import MarkerCluster

# Let's first create a MarkerCluster object
marker_cluster = MarkerCluster()

# Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster
    marker_cluster.add_child(marker)

site_map

# Observations:

    - KSC LC-39A has highest number of class 1 vs class 0 launches
    - CCAFS SLC-40 has the highest number of class 0 vs class 1 launches
    

# TASK 3: Calculate the distances between a launch site to its proximities

In [28]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [39]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon) omitted the coastline string because... i will use this for everything.
def add_distance_marker(launch_lat, launch_lon, target_lat, target_lon):
    distance = calculate_distance(launch_lat, launch_lon, target_lat, target_lon)
    marker = folium.Marker(
        location=[target_lat, target_lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
    site_map.add_child(marker)

    
    line = folium.PolyLine(
        locations=[[launch_lat, launch_lon], [target_lat, target_lon]],
        weight=2,
        color='blue'
    )
    site_map.add_child(line)


## I don't want to repeat the code block for every site, so I turned it to a function so I can just call it on the coordinates

In [42]:
city_lat, city_lon = 29.68775, -95.21027  
rail_lat, rail_lon = 29.53717, -95.11774   
highway_lat, highway_lon = 29.52511, -95.12748  


# Launch site coordinates (NASA JSC main site)
launch_lat, launch_lon = 29.55935, -95.08332  

# City
add_distance_marker(launch_lat, launch_lon, city_lat, city_lon)

# Railroad
add_distance_marker(launch_lat, launch_lon, rail_lat, rail_lon)

# Highway
add_distance_marker(launch_lat, launch_lon, highway_lat, highway_lon)

site_map